In [1]:
import cobra
from cobra.io.mat import *
import warnings
warnings.simplefilter('ignore')
#from cobra.sampling import sample
from cobra.sampling import OptGPSampler, ACHRSampler
import os
cpus = os.cpu_count()
from cobra.io.json import load_json_model
#model = load_matlab_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.mat")
model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [2]:
print(f'Number of reactions: {model.reactions.__len__()}')
print(f'Number of metabolites: {model.metabolites.__len__()}')
print(f'Number of compartments: {model.compartments.__len__()}')

Number of reactions: 10602
Number of metabolites: 5835
Number of compartments: 9


In [3]:
model.objective.expression

1.0*biomass - 1.0*biomass_reverse_01e59

In [4]:

#recon301 =load_matlab_model("./COBRA_models/Recon3DModel_301.mat")
model.reactions.get_by_id('biomass_maintenance').bounds = 0, 0
model.reactions.get_by_id('biomass').bounds             = .5*model.optimize().objective_value, 100
model.objective = 'r0399'
model.reactions.get_by_id("r0399").bounds = (0, 1e5)
model.reactions.get_by_id("PHETHPTOX2").bounds = (0, 0.0)

In [5]:
def turn_off_rxn(rxn_id):
    model.reactions.get_by_id(rxn_id).bounds       = (0, 0)
   
    return model.reactions.get_by_id(rxn_id).bounds 
    


list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("tyr_L_e").reactions]))
list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("dhbpt_e").reactions]))
list(map(turn_off_rxn, [r.id for r in model.metabolites.get_by_id("thbpt_e").reactions]))

[(0, 0), (0, 0)]

In [6]:
import itertools


regulatory_reactions = model.optimize().reduced_costs.loc[lambda x: abs(x)>1e-12].index.tolist()
regulatory_reactions = list(
itertools.compress(regulatory_reactions, 
                   np.invert(abs(np.array([list(model.reactions.get_by_id(rr).bounds) for rr in regulatory_reactions]
)).sum(axis=1) == 0)))
      
[model.reactions.get_by_id(rr).reaction for rr in regulatory_reactions]

['5mthf_c + dhbpt_c --> mlthf_c + thbpt_c',
 'thbpt4acam_c --> dhbpt_c + h2o_c',
 'o2_c + thbpt_c + trp_L_c --> 5htrp_c + dhbpt_c + h2o_c',
 'dhbpt_c + 2.0 h_c + nadh_c --> nad_c + thbpt_c',
 'dhbpt_c + 2.0 h_c + nadph_c --> nadp_c + thbpt_c',
 'o2_c + thbpt_c + tyr_L_c --> 34dhphe_c + dhbpt_c + h2o_c']

In [7]:
def set_rxn_bounds(rxn_id):
    model.reactions.get_by_id(rxn_id).bounds       = (0, 33.5)
    
    return model.reactions.get_by_id(rxn_id).bounds 


list(map(set_rxn_bounds, regulatory_reactions))

model.optimize().objective_value

100.5

In [8]:
from cobra.sampling import OptGPSampler


##HEALTHY
model.reactions.get_by_id("r0399").bounds       = (99, model.optimize().objective_value)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (0, 0)



optgp = OptGPSampler(model, processes=os.cpu_count(), thinning=500)
samples = optgp.sample(20_000)
samples.to_parquet("./results/fluxes/flux_samples_CONTROL_20_000.parquet.gzip", compression='gzip')  

In [9]:
##PKU
model.reactions.get_by_id("r0399").bounds       = (0, 5)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (0, 0)



optgp = OptGPSampler(model, processes=os.cpu_count(), thinning=500)
samples_pku = optgp.sample(20_000)
samples_pku.to_parquet("./results/fluxes/flux_samples_PKU_20_000.parquet.gzip", compression='gzip')  